Neural Network.

Adapted from https://www.kaggle.com/datasets/johnsmith88/heart-disease-dataset


In [ ]:
# GLOBALS
LEARNING_RATE = 0.001 # 0.001, 0.003, 0.03
using_heart = True # True, False

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

# Load Datasets
from google.colab import drive
drive.mount('/content/drive')
if (using_heart):
  dataset = np.genfromtxt('/content/drive/MyDrive/datasets/heart.csv', dtype=float, delimiter=',', skip_header=1)
else:
  dataset = np.loadtxt('/content/drive/MyDrive/datasets/pima-indians-diabetes.data.csv', delimiter=',')

dataset

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


array([[52.,  1.,  0., ...,  2.,  3.,  0.],
       [53.,  1.,  0., ...,  0.,  3.,  0.],
       [70.,  1.,  0., ...,  0.,  3.,  0.],
       ...,
       [47.,  1.,  0., ...,  1.,  2.,  0.],
       [50.,  0.,  0., ...,  0.,  2.,  1.],
       [54.,  1.,  0., ...,  1.,  3.,  0.]])

##TODOS:
####Antes de tener resultados:
1. Revisar si los datos están balanceados en ambos datasets.
2. Agregar F1-Score
3. Correr las 4 versiones de ambas redes (con lr = 0.001, 0.003 ,0.03)
- 3.1 Input -> 12 -> 1 MaevaWinter
- 3.2 Input -> 24 -> 1 Joha-neural-network
- 3.3 Input -> 8 -> 1 Randalistico
- 3.4 Input -> 12 -> 24 -> 1 Joe

####Después de tener resultados:
1. EDA


####Notas
Pima-diabetes
268 of 768 are 1, the others are 0

Heart
526 of 1025 are 1, the others are 0

In [ ]:
# Split into input (X) and output (y) variables
if (using_heart):
  X = dataset[:,0:13]
  y = dataset[:,13]

  X = torch.tensor(X, dtype=torch.float32)
  y = torch.tensor(y, dtype=torch.float32).reshape(-1, 1)

  print(X)

else:
  X = dataset[:,0:8]
  y = dataset[:,8]

  # Oversampling
  # We are only over sampling over training data in order to
  # predict only over the real values
  from imblearn.over_sampling import SMOTE
  oversample = SMOTE()
  X_resampled, y_resampled = oversample.fit_resample(X, y)

  # Train test sets to torch tensors.
  X_resampled = torch.tensor(X, dtype=torch.float32)
  y_resampled = torch.tensor(y, dtype=torch.float32).reshape(-1, 1)

  X = torch.tensor(X, dtype=torch.float32)
  y = torch.tensor(y, dtype=torch.float32).reshape(-1, 1)

  print(X, X_resampled)

tensor([[52.,  1.,  0.,  ...,  2.,  2.,  3.],
        [53.,  1.,  0.,  ...,  0.,  0.,  3.],
        [70.,  1.,  0.,  ...,  0.,  0.,  3.],
        ...,
        [47.,  1.,  0.,  ...,  1.,  1.,  2.],
        [50.,  0.,  0.,  ...,  2.,  0.,  2.],
        [54.,  1.,  0.,  ...,  1.,  1.,  3.]])


In [ ]:
# define the model
if (using_heart):
  model = nn.Sequential(
      nn.Linear(13, 12),
      nn.ReLU(),
      nn.Linear(12, 24),
      nn.ReLU(),
      nn.Linear(24, 1),
      nn.Sigmoid()
  )
else:
  model = nn.Sequential(
      nn.Linear(8, 12),
      nn.ReLU(),
      nn.Linear(12, 24),
      nn.ReLU(),
      nn.Linear(24, 1),
      nn.Sigmoid()
  )
print(model)

Sequential(
  (0): Linear(in_features=13, out_features=12, bias=True)
  (1): ReLU()
  (2): Linear(in_features=12, out_features=24, bias=True)
  (3): ReLU()
  (4): Linear(in_features=24, out_features=1, bias=True)
  (5): Sigmoid()
)


In [ ]:
# train the model
loss_fn   = nn.BCELoss()  # binary cross entropy
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

n_epochs = 100
batch_size = 10

## This if is redundant if yo rename x_resampled and y_resampled
# to the simpler X and Y, but it is left separate on purpose for
# studying reasons.
if (using_heart):
  for epoch in range(n_epochs):
      for i in range(0, len(X), batch_size):
          Xbatch = X[i:i+batch_size]
          y_pred = model(Xbatch)
          ybatch = y[i:i+batch_size]
          loss = loss_fn(y_pred, ybatch)
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()
      print(f'Finished epoch {epoch}, latest loss {loss}')

else:
  for epoch in range(n_epochs):
      for i in range(0, len(X), batch_size):
          Xbatch = X_resampled[i:i+batch_size]
          y_pred = model(Xbatch)
          ybatch = y_resampled[i:i+batch_size]
          loss = loss_fn(y_pred, ybatch)
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()
      print(f'Finished epoch {epoch}, latest loss {loss}')

Finished epoch 0, latest loss 0.7132564187049866
Finished epoch 1, latest loss 0.593629777431488
Finished epoch 2, latest loss 0.5214617252349854
Finished epoch 3, latest loss 0.4637835919857025
Finished epoch 4, latest loss 0.41851943731307983
Finished epoch 5, latest loss 0.38422760367393494
Finished epoch 6, latest loss 0.3520282804965973
Finished epoch 7, latest loss 0.3257197439670563
Finished epoch 8, latest loss 0.3016853332519531
Finished epoch 9, latest loss 0.2786504328250885
Finished epoch 10, latest loss 0.2584880590438843
Finished epoch 11, latest loss 0.23949286341667175
Finished epoch 12, latest loss 0.2233273684978485
Finished epoch 13, latest loss 0.20906344056129456
Finished epoch 14, latest loss 0.1969996988773346
Finished epoch 15, latest loss 0.18517862260341644
Finished epoch 16, latest loss 0.17790380120277405
Finished epoch 17, latest loss 0.16806906461715698
Finished epoch 18, latest loss 0.1634366512298584
Finished epoch 19, latest loss 0.15682142972946167
Fin

In [ ]:
# compute accuracy (no_grad is optional)
with torch.no_grad():
    y_pred = model(X)
accuracy = (y_pred.round() == y).float().mean()
print(f"Accuracy {accuracy}")

from sklearn.metrics import f1_score
y_np = y.numpy()
y_pred_np = y_pred.round().numpy()
f1 = f1_score(y_np, y_pred_np)
print(f"F1_score {f1}")

Accuracy 0.821463406085968
F1_score 0.8455696202531646


In [ ]:
## For quicker experimental runs.

# define the model

for experiment in range(9):

  if (using_heart):
    model = nn.Sequential(
        nn.Linear(13, 12),
        nn.ReLU(),
        nn.Linear(12, 24),
        nn.ReLU(),
        nn.Linear(24, 1),
        nn.Sigmoid()
    )
  else:
    model = nn.Sequential(
        nn.Linear(8, 12),
        nn.ReLU(),
        nn.Linear(12, 24),
        nn.ReLU(),
        nn.Linear(24, 1),
        nn.Sigmoid()
    )
  #print(model)
  # train the model
  loss_fn   = nn.BCELoss()  # binary cross entropy
  optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

  n_epochs = 100
  batch_size = 10

  if (using_heart):
    for epoch in range(n_epochs):
        for i in range(0, len(X), batch_size):
            Xbatch = X[i:i+batch_size]
            y_pred = model(Xbatch)
            ybatch = y[i:i+batch_size]
            loss = loss_fn(y_pred, ybatch)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        #print(f'Finished epoch {epoch}, latest loss {loss}')

  else:
    for epoch in range(n_epochs):
        for i in range(0, len(X), batch_size):
            Xbatch = X_resampled[i:i+batch_size]
            y_pred = model(Xbatch)
            ybatch = y_resampled[i:i+batch_size]
            loss = loss_fn(y_pred, ybatch)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        #print(f'Finished epoch {epoch}, latest loss {loss}')

  # compute accuracy (no_grad is optional)
  with torch.no_grad():
      y_pred = model(X)
  accuracy = (y_pred.round() == y).float().mean()
  print(f"Accuracy {accuracy}")

  from sklearn.metrics import f1_score
  y_np = y.numpy()
  y_pred_np = y_pred.round().numpy()
  f1 = f1_score(y_np, y_pred_np)
  print(f"F1_score {f1}")

Accuracy 0.8390243649482727
F1_score 0.8586118251928021
Accuracy 0.8390243649482727
F1_score 0.8561464690496948
Accuracy 0.8390243649482727
F1_score 0.8578811369509044
Accuracy 0.8604878187179565
F1_score 0.873114463176575
Accuracy 0.8321951031684875
F1_score 0.8501742160278746
Accuracy 0.8292682766914368
F1_score 0.8497854077253219
Accuracy 0.8399999737739563
F1_score 0.8593481989708405
Accuracy 0.8468292951583862
F1_score 0.8638334778837815
Accuracy 0.8370731472969055
F1_score 0.8554112554112555
